# Step 2 — FEATURE ENGINEERING (LLM-AIDED)

## 🎯 Objective

Transform the **frozen dataset** into a **model-ready feature table**, using:
* Encoding
* Derived variables
* **LLM/SLM-assisted text feature extraction**
* Target variable definition

## 1️⃣ Input (DO NOT CHANGE)

Person B **must not regenerate or modify raw data**.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/synthetic_customers_raw.csv")
df.head()

## 2️⃣ Decide Feature Categories (Design First)

### Feature Types

| Type | Examples |
| --- | --- |
| **Numerical (Raw)** | `age`, `income`, `total_orders`, `avg_order_value`, `days_since_last_purchase` |
| **Derived Numerical** | `income_per_order` |
| **LLM-Based Text Features** | `sentiment_score`, `risk_score` (from `review_text`) |
| **Target Variable** | `churn` (Derived from inactivity rules) |

## 3️⃣ LLM-AIDED TEXT FEATURE EXTRACTION

We use an LLM design to classify unstructured `review_text` into structured features.

### Step 3A — LLM Prompt (Documentation)

```text
You are a customer sentiment analyst.
Given short e-commerce reviews, classify them into:
1. Sentiment: Positive / Neutral / Negative
2. Churn Risk: Low / Medium / High

Provide keyword-based rules for Python implementation.
```

### LLM Output (Summarised Rule Set)
- **Positive / Low Risk**: "satisfied", "excellent", "recommended", "great", "buy again"
- **Negative / High Risk**: "disappointed", "poor", "slow", "bad", "not worth"
- **Neutral / Medium Risk**: Words like "okay", "average", "acceptable"

In [ ]:
## Step 3B — Implement LLM Rules in Python

def extract_sentiment_and_risk(text):
    text = text.lower()
    
    # Keywords derived from LLM suggestions
    negative_keywords = ["disappointed", "poor", "slow", "bad", "not worth"]
    positive_keywords = ["excellent", "satisfied", "recommended", "great", "buy again"]

    if any(word in text for word in negative_keywords):
        return "negative", "high"
    elif any(word in text for word in positive_keywords):
        return "positive", "low"
    else:
        return "neutral", "medium"

df[["sentiment", "churn_risk"]] = df["review_text"].apply(
    lambda x: pd.Series(extract_sentiment_and_risk(x))
)

df[["review_text", "sentiment", "churn_risk"]].head()

## 4️⃣ Encode Features & Define Target

### Target Definition: Churn
Since the raw data doesn't have a label, we derive it using business logic:
* **Churned (1)**: Inactive for > 180 days
* **Active (0)**: Inactive for <= 180 days

In [ ]:
# 1. Define Target
df["churn"] = (df["days_since_last_purchase"] > 180).astype(int)

# 2. Ordinal Encoding for LLM Features
sentiment_map = {"negative": 0, "neutral": 1, "positive": 2}
risk_map = {"low": 0, "medium": 1, "high": 2}

df["sentiment_score"] = df["sentiment"].map(sentiment_map)
df["risk_score"] = df["churn_risk"].map(risk_map)

# 3. Derived Features
df["income_per_order"] = df["income"] / df["total_orders"]

df.head()

## 5️⃣ Prepare Final Feature Set (FOR MODELS)

In [ ]:
features = df[[
    "age", "income", "total_orders", "avg_order_value", 
    "days_since_last_purchase", "income_per_order", 
    "sentiment_score", "risk_score"
]]

target = df["churn"]

features.to_csv("../data/features.csv", index=False)
target.to_csv("../data/target.csv", index=False)

print("Features and Target saved successfully.")

## 6️⃣ Document AI Usage (Assessment Requirement)

> **AI-Assisted Feature Engineering**
>
> A Large Language Model was used to design rule-based mappings for extracting **sentiment** and **churn risk** from unstructured review texts. 
> These AI-informed rules were implemented in Python to derive ordinal numerical features (`sentiment_score`, `risk_score`) used in predictive modelling.